In [1]:
import pandas as pd
import numpy as np
import math
import os
import datetime 
import qgrid
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

C:\Users\the7490\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\the7490\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1 Data Preprocessing
## 1.1 Read Files

In [2]:
# --- SAP VA05 File ---
#File path
va_file_dir = r'C:\Users\the7490\MDLZ\Central Analytics Team CAT - Projects\Beta Quadrant\KUNLUN\01.Raw Data\VA05_CSV'

#get file name list
va_file_list = os.listdir(va_file_dir)
new_list = []


for file in va_file_list:
    #construct file path
    
    file_path = os.path.join(va_file_dir,file)
    
    df = pd.read_csv(file_path , encoding = 'utf-8',thousands= r',')
    
    df['Doc. Date'] = pd.to_datetime(df['Doc. Date'])

    df = df.rename(columns=lambda x: x.strip())
    
    new_list.append(df)
    # print(file)
    
#Combine data
va = pd.concat(new_list)

# --- CRS data ---
# file_dir = r'C:\Users\the7490\Documents\01 RDS\data\crs_order'
# file_list = os.listdir(file_dir)
# new_list = []

# for file in file_list:
#     file_path = os.path.join(file_dir,file)
#     dataframe = pd.read_csv(file_path)
#     new_list.append(dataframe)
    
# #Combine data
# crs = pd.concat(new_list)
crs = pd.read_csv(r'C:\Users\the7490\Downloads\crs_updated_following.csv',encoding='utf-8')

# --- Boundary data ---
file_dir = r'C:\Users\the7490\Documents\01 RDS\data\min_max_intentory_date'


file_list = os.listdir(file_dir)
new_list = []

for file in file_list:
    file_path = os.path.join(file_dir,file)
    dataframe = pd.read_csv(file_path)
    new_list.append(dataframe)

    
boundary = pd.concat(new_list)

# SKU mapping - Master
sku_mapping = pd.read_excel(r'C:\Users\the7490\Documents\01 RDS\data\SKU Category Mapping List V2.xlsx', engine = "xlrd")
sku_mapping = sku_mapping[['sku_code','Category']].astype(str)

## Functions

In [3]:
def qual_quant_features(data):
    dtypes = list(map(lambda x:str(x),list(data.dtypes)))
    qualitative = []
    quantitative = []
    for i in range(len(dtypes)):
        if dtypes[i] == 'object':
            qualitative.append(data.columns[i])
        else:
            quantitative.append(data.columns[i])
    return qualitative,quantitative

qualitative,quantitative = qual_quant_features(va)

# Create a function to show result
def result_show(path):
    return qgrid.show_grid(path,show_toolbar = True)

def trim(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) #trim
    return df

### 1.2.1 Clean Data - SAP VA05

In [4]:
# SAP处理考虑int
# Filter out wanted columns and set data types
va_df = va[['Rj','PO Number','Sold-To Pt','Name 1','Material','Description','Plnt','Order Qty','ConfirmQty','Doc. Date','Cust.price','Net price','SaTy']]
# va_df = va_df[va_df['Order Qty']>0]

# va_df[['Sold-To Pt','Material']] = va_df[['Sold-To Pt','Material']].astype(float).astype(int).astype(str)
# va_df = va_df[~va_df['Material'].str.contains(r'DM')]
# va_df[['Material']] = va_df[['Material']].astype(float)

# Remove na material rows
# va_df = va_df[~va_df['Material'].isna()]

va_df[['Material','Sold-To Pt']] = va_df[['Material','Sold-To Pt']].astype(str)

va_df[['Order Qty','ConfirmQty']] = va_df[['Order Qty','ConfirmQty']].fillna(0).astype(int)

# Saty == ZOR  and PO number start with VZO for CRS customer
va_df =  va_df[va_df['SaTy']=='ZOR']
va_df = va_df[va_df['PO Number'].str.contains(r'VZO*')]

# Replace null with 0 for Rj
va_df['Rj'] = va_df['Rj'].fillna(0).astype(int)

# trim
va_df = trim(va_df)

# Remove 61 code
va_df = va_df[va_df['Rj']!= 61]

# Set Rj=10 if order_qty > confrimQty and Rj is null
# va_df['Rj'] = va_df[['Order Qty','ConfirmQty','Rj']].apply(lambda x: 10 if x['Rj']== 0 and x['Order Qty']> x['ConfirmQty'] else x['Rj'], axis =1)

# Gourp by Customer, Material, Date to map CRS data and count 
va_df_grouped = va_df[['Order Qty','ConfirmQty','Sold-To Pt','Material','Doc. Date','Rj']].groupby(["Sold-To Pt",'Material','Doc. Date']).agg({'Order Qty':'sum','ConfirmQty':'sum','Rj':'count'}).reset_index()
va_df_grouped.rename(columns ={'Rj':'count'}, inplace = True)


# Get Customer and SKU Info
va_cus = va_df[["Sold-To Pt",'Name 1','Plnt']].drop_duplicates(subset=['Sold-To Pt'],keep='first',inplace=False)
va_df_grouped_cus = pd.merge(va_df_grouped, va_cus, on = 'Sold-To Pt',how = 'left')

va_sku = va_df[["Material",'Description','Net price']].drop_duplicates(subset=['Material'],keep='first',inplace=False)
df_va = pd.merge(va_df_grouped_cus, va_sku, on = 'Material',how = 'left')

# df_va =  pd.merge(df_va, date[['Date','year','week']], left_on = 'Doc. Date',right_on = 'Date',how = 'left') 
df_va['year'] = df_va['Doc. Date'].dt.year.astype(int)
df_va['week'] = df_va['Doc. Date'].apply(lambda x: x.strftime("%W")).astype(int) # Start from Monday
df_va['ds'] = df_va['Doc. Date'].apply(lambda x: x.strftime("%Y%m%d")).astype(str)

In [ ]:
df_va['Order Qty'].sum()

### 1.2.2 Clean data - CRS 

In [5]:
# drop unwanted_columns
crs = crs.drop(columns = ['Unnamed: 0','year','week'])

# Replace \N with null
crs = crs.replace({r'\N': None})

# Set data type
crs['calc_date'] = pd.to_datetime(crs['calc_date'])
crs[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']] = crs[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5']].astype(float)
crs[['sale_week','sale_week1','sale_week2','sale_week3','sale_week4']] = crs[['sale_week','sale_week1','sale_week2','sale_week3','sale_week4']].astype(float)

crs[['sold_to_pt','material','ds']] = crs[['sold_to_pt','material','ds']].astype(str)
crs[['order_qty','pgi_qty']] = crs[['order_qty','pgi_qty']].astype(int)

crs = trim(crs)

### 1.2.3 Clean data - boundary

In [6]:
boundary.drop_duplicates(subset=['CategoryName','ReceiverCode'], keep='first',inplace=True)
boundary = boundary.drop(columns=['CategoryCode','HotFlag','ReceiverName', 'InsertTime'])
boundary[['ReceiverCode']] = boundary[['ReceiverCode']].astype(str)
boundary = trim(boundary)

#####  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1.3 Merge tables

In [8]:
# merge crs and va
va_crs = pd.merge(df_va, crs,left_on=['Sold-To Pt','Material','ds'],right_on=['sold_to_pt','material','ds'], how='left')

# merge sku mapping to get category 
va_crs_sku = pd.merge(va_crs, sku_mapping,left_on=['material'],right_on=['sku_code'],how='left')

#  merge boundary
va_crs_sku_bdy = pd.merge(va_crs_sku, boundary,left_on=['sold_to_pt','Category'],right_on=['ReceiverCode','CategoryName'],how='left')

## 1.4 Organize data 

In [9]:
df_prep = va_crs_sku_bdy

# Fill NA min as 14 and max as 28
df_prep['MinDays'] = df_prep['MinDays'].replace(np.nan, 14).astype(int)
df_prep['MaxDays'] = df_prep['MaxDays'].replace(np.nan, 28).astype(int)

# keep records of ordered date only
df_prep = df_prep[df_prep['order_qty']>0]
# df_prep = df_prep[df_prep['Order Qty']>0]


# remove avg_sale < 0
df_prep = df_prep[df_prep['avg_sales_box_day']>0]

# remove duplicated columns
df_prep = df_prep.drop(columns=['sold_to_pt','material','sku_code','ReceiverCode','calc_date'])

# set data type and  replace fill na with 0
df_prep[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5','sale_week1','sale_week2',
         'sale_week3','sale_week4','sale_week']] = \
df_prep[['suggest_box','sale_box_week1','sale_box_week2','sale_box_week3','sale_box_week4','sale_box_week5','sale_week1','sale_week2','sale_week3','sale_week4','sale_week']].fillna(0).astype(float)

# df_prep['sale_month'] = (df_prep['sale_box_week1']+ df_prep['sale_box_week2']+ df_prep['sale_box_week3']+ df_prep['sale_box_week4'])
df_prep['sale_month'] = (df_prep['sale_week1']+ df_prep['sale_week2']+ df_prep['sale_week3']+ df_prep['sale_week4'])

# rename columns 
df_prep = df_prep.rename(columns = {'Sold-To Pt':'sold_to_pt','Material':'material'})

In [10]:
df_prep['Order Qty'].sum()

27073617

In [11]:
df_prep['MaxDays'] = df_prep['MaxDays']*1.8

## 1.5 Tag - CRS + KA SAP CFR 94%

In [12]:
# Apply shortage tag via CRS data including reason code 61 (customer real need)
# SAP data starts from 2020
crs['year'] = crs['calc_date'].dt.year.astype(int)
crs['week'] = crs['calc_date'].apply(lambda x: x.strftime("%W")).astype(int)   #Start from Monday
crs_incl_61 = crs

crs_incl_61 = crs_incl_61[['order_qty','pgi_qty','year','week','material']].groupby(['year','week','material']).sum().reset_index()

# CFR < 94% → shorage
crs_incl_61['Shortage_incl_61'] = crs_incl_61[['order_qty','pgi_qty']].apply(lambda x: 'shortage' if x['pgi_qty']/x['order_qty'] < 0.94 else 'normal', axis = 1)

# merge
# df_prep = pd.merge(df_prep, crs_incl_61[['year','week','material','Shortage_incl_61']], on = ['year','week','material'],how = 'left')

In [13]:
df_prep['Order Qty'].sum()

27073617

In [14]:
# Define order and pgi type
def assign_order_type(df):
    order = df['Order Qty']
    inv = df['avail_inventory_box']
    ts = df['intrans_inventory_box']
    mx = df['MaxDays']
    mi = df['MinDays']
    avg = df['avg_sales_box_day']
    
    if order+inv +ts > mx*avg :
        return 'over'
    elif order+inv + ts < mi*avg :
        return 'under'
    else: # order+inv between(mi*avg, mx*avg)
        return 'normal'
    
def assign_pgi_type(df):
    pgi = df['ConfirmQty']
    inv = df['avail_inventory_box']
    ts = df['intrans_inventory_box']
    mx = df['MaxDays']
    mi = df['MinDays']
    avg = df['avg_sales_box_day']
    
    if pgi+inv +ts > mx*avg :
        return 'exceeded'
    elif pgi+inv +ts < mi*avg :
        return 'lacking'
    else: # pgi+inv between(mi*avg, mx*avg)
        return 'normal'
    
df_prep['order_type'] = df_prep.apply(assign_order_type,axis=1)
df_prep['pgi_type'] = df_prep.apply(assign_pgi_type,axis=1)

#df_prep.rename(columns = {'Shortage_incl_61': 'shortage'}, inplace = True)

In [15]:
df_crs = df_prep

## *1.6 Add KA data

In [16]:
ka_cus = pd.read_csv(r'C:\Users\the7490\Documents\01 RDS\data\dim_customer_atom.csv',encoding='gbk')


ka = ka_cus[['sold_to_code','ka_type_bc']].astype(str)
ka = ka[ka['ka_type_bc'].str.contains(r'NKA|LKA',regex=True)]
ka_list =ka[['sold_to_code']].drop_duplicates().astype(int)

temp_list = []

for cus in zip(ka_list['sold_to_code']):
    temp = va[va['Sold-To Pt']==cus]
    temp_list.append(temp)
    
va_ka = pd.concat(temp_list)

# keep records of ordered date only
va_ka = va_ka[va_ka['Order Qty']>0]

va_ka['Rj'] = va_ka['Rj'].fillna(0).astype(int)

va_ka[['Sold-To Pt','Material']] = va_ka[['Sold-To Pt','Material']].astype(float).astype(int).astype(str)

va_ka = trim(va_ka)

# remove 61!!!
va_ka = va_ka[va_ka['Rj']!= 61]

# Remove na material rows
va_ka = va_ka[~va_ka['Material'].isna()]

va_ka[['Material','Sold-To Pt']] = va_ka[['Material','Sold-To Pt']].astype(str)

# Gourp by Customer, Material, Date to map CRS data and count 
va_ka_grouped = va_ka[['Order Qty','ConfirmQty','Sold-To Pt','Material','Doc. Date','PO Number']].groupby(["Sold-To Pt",'Material','Doc. Date']).agg({'Order Qty':'sum','ConfirmQty':'sum','PO Number':'count'}).reset_index()
va_ka_grouped.rename(columns ={'PO Number':'count'}, inplace = True)


# Get Customer and SKU Info
va_ka_cus = va_ka[["Sold-To Pt",'Name 1','Plnt']].drop_duplicates(subset=['Sold-To Pt'],keep='first',inplace=False)
va_ka_info = pd.merge(va_ka_grouped, va_ka_cus, on = 'Sold-To Pt',how = 'left')

va_ka_sku = va_ka[["Material",'Description','Net price']].drop_duplicates(subset=['Material'],keep='first',inplace=False)
df_ka = pd.merge(va_ka_info, va_ka_sku, on = 'Material',how = 'left')

# df_va =  pd.merge(df_va, date[['Date','year','week']], left_on = 'Doc. Date',right_on = 'Date',how = 'left') 
df_ka['year'] = df_ka['Doc. Date'].dt.year.astype(int)
df_ka['week'] = df_ka['Doc. Date'].apply(lambda x: x.strftime("%W")).astype(int)    # Start from Monday  
df_ka['ds'] = df_ka['Doc. Date'].apply(lambda x: x.strftime("%Y%m%d")).astype(int)

In [17]:
df_ka['Order Qty'].sum()

18468161.484

In [19]:
# tag update
df_ka_tag = df_ka
df_ka_tag.rename(columns = {'Sold-To Pt':'sold_to_pt','Material':'material'}, inplace = True)
df_ka_tag[['sold_to_pt','material']] = df_ka_tag[['sold_to_pt','material']].astype(str)

# Tag pgi type
df_ka_tag['pgi_type'] = df_ka_tag[['Order Qty','ConfirmQty']].apply(lambda x: 'lacking' if x['Order Qty']>x['ConfirmQty'] else 'normal',axis=1)

# Do not assign order type for KA as we don't sugget order qty for them

In [20]:
## *1.7 Append 
df_crs['CusType'] = 'CRS'
df_ka_tag['CusType']='KA'
df = df_crs.append(df_ka_tag)

In [21]:
va_ka_tag = pd.concat(temp_list)

# keep records of ordered date only
va_ka_tag = va_ka_tag[va_ka_tag['Order Qty']>0]

va_ka_tag['Rj'] = va_ka_tag['Rj'].fillna(0).astype(int)

va_ka_tag[['Sold-To Pt','Material']] = va_ka_tag[['Sold-To Pt','Material']].astype(float).astype(int).astype(str)

va_ka_tag = trim(va_ka_tag)


# Remove na material rows
va_ka_tag = va_ka_tag[~va_ka_tag['Material'].isna()]

va_ka_tag[['Material','Sold-To Pt']] = va_ka_tag[['Material','Sold-To Pt']].astype(str)

va_ka_tag.rename(columns = {'Sold-To Pt':'sold_to_pt','Material':'material'}, inplace = True)
va_ka_tag[['sold_to_pt','material']] = va_ka_tag[['sold_to_pt','material']].astype(str)
# # Gourp by Customer, Material, Date to map CRS data and count 
# va_ka_grouped = va_ka[['Order Qty','ConfirmQty','Sold-To Pt','Material','Doc. Date','PO Number']].groupby(["Sold-To Pt",'Material','Doc. Date']).agg({'Order Qty':'sum','ConfirmQty':'sum','PO Number':'count'}).reset_index()
# va_ka_grouped.rename(columns ={'PO Number':'count'}, inplace = True)


# # Get Customer and SKU Info
# va_ka_cus = va_ka[["Sold-To Pt",'Name 1','Plnt']].drop_duplicates(subset=['Sold-To Pt'],keep='first',inplace=False)
# va_ka_info = pd.merge(va_ka_grouped, va_ka_cus, on = 'Sold-To Pt',how = 'left')

# va_ka_sku = va_ka[["Material",'Description','Net price']].drop_duplicates(subset=['Material'],keep='first',inplace=False)
# df_ka = pd.merge(va_ka_info, va_ka_sku, on = 'Material',how = 'left')

# # df_va =  pd.merge(df_va, date[['Date','year','week']], left_on = 'Doc. Date',right_on = 'Date',how = 'left') 
va_ka_tag['year'] = va_ka_tag['Doc. Date'].dt.year.astype(int)
va_ka_tag['week'] = va_ka_tag['Doc. Date'].apply(lambda x: x.strftime("%W")).astype(int)    # Start from Monday  
# df_ka['ds'] = df_ka['Doc. Date'].apply(lambda x: x.strftime("%Y%m%d")).astype(int)

### 1.7 Shoratge tag update

In [22]:
crs_incl_61 = crs_incl_61.rename(columns = {'order_qty':'Order Qty','pgi_qty':'ConfirmQty'})
crs_incl_61['CusType'] = 'CRS'
df_tag = crs_incl_61.append(va_ka_tag)
df_tag = df_tag[df_tag['Order Qty']>0]

In [23]:
df_tag = df_tag[['Order Qty','ConfirmQty','year','week','material']].groupby(['year','week','material']).sum().reset_index()

# CFR < 94% → shorage
df_tag['shortage'] = df_tag[['Order Qty','ConfirmQty']].apply(lambda x: 'shortage' if x['ConfirmQty']/x['Order Qty'] < 0.94 else 'normal', axis = 1)

# tag 
df = pd.merge(df, df_tag[['year', 'week', 'material', 'shortage']], on=['year','week','material'],how='left')

In [24]:
# merge crs by 94% cfr shortage tag
df = pd.merge(df, crs_incl_61[['year','week','material','Shortage_incl_61']], on = ['year','week','material'],how = 'left')

In [25]:
df['shortage'].unique()

array(['normal', 'shortage'], dtype=object)

# 2 Allocation 
## 2.1 Calculate Gap and exceed amount for shortage records

In [26]:
# Exceed amount: take both max boundary and following month sale into consideration 
    # exceed box = pgi - avail_inventory - max( MaxBoundary* avg_sales, sales_month)
    # Add transitb
exceed_pgi_box = []

for ot,pt,stg,order,pgi,inv,mx,sale,avg,ts in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['sale_month'],df['avg_sales_box_day'],df['intrans_inventory_box']):
    if stg == 'shortage' and ot == 'over' and pt == 'exceeded':
        if inv + ts > mx*avg:  # Case a:  consider max boundary
            ex_a = pgi #exceed = pgi 
        elif inv+pgi +ts <=mx*avg:
            ex_a = 0 # no exceed pgi
        else: 
            ex_a = pgi + inv +ts - mx*avg
            
        if pgi+inv > sale: # Case b: consider sales in following month,未来一个月的情况考虑在途不公平，在途不能立马转销售
            ex_b = pgi+inv-sale # not sell in a month
        elif pgi+inv <= sale: #  pgi+inv <= sale sell in a month
            ex_b = 0
        else:
            ex_b = None
        exceed_pgi_box.append(int(min(ex_a,ex_b)))
    else:
        exceed_pgi_box.append(None)

        
df['exceed_pgi_box'] = exceed_pgi_box 

df['exceed_pgi_box'].sum()  #3,058,461  / 2,601,679 / transit 3,209,263

2391939.0

In [27]:
df['Order Qty'].sum()

45541778.484000005

## * Add KA gap for crs shortage sku, gap = order - pgi

In [28]:
# scenario 1： 
    # normal/under ordering GAP = order - pgi - avail_inventory - transit
    # over orderging and !exceeded pgi type GAP = max boundary * avg_sales, sale_month - pgi - avail_inventory - transit
    # 考虑transit
gap_box_s1 = []

for ot,pt,stg,order,pgi,inv,mx,mi,avg,cus,ts in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day'],df['CusType'],df['intrans_inventory_box']):
    if stg == 'shortage'and cus == 'CRS':
        if ot != 'over':
            gap_box_s1.append(round(max(order - pgi - inv -ts,0)))
        elif ot == 'over' and pt !='exceeded': 
            gap_box_s1.append(round(max(mx*avg - pgi - inv -ts,0)))
        else:        
            gap_box_s1.append(None)
    elif stg == 'shortage' and cus == 'KA':
        gap_box_s1.append(round(order-pgi))
    else:
        gap_box_s1.append(None)
        
df['gap_box_s1'] = gap_box_s1
df['gap_box_s1'].sum() # 443,076 -> 272162

2408236.0

In [29]:
df[df['CusType']=='CRS'].gap_box_s1.sum()

335848.0

In [30]:
# scenario 2： 
    # normal GAP = min*avg - pgi - avail_inventory
    # under ordering GAP = order - pgi - avail_inventory
    # over ordering and !exceeded pgi GAP = min*avg - pgi - avail_inventory
gap_box_s2= []

for ot,pt,stg,order,pgi,inv,mx,mi,avg,cus,ts in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day'],df['CusType'],df['intrans_inventory_box']):
    if stg == 'shortage' and cus == 'CRS':
        if ot == 'normal':
            gap_box_s2.append(round(max(mi*avg - pgi - inv -ts,0)))
        elif ot == 'under': 
            gap_box_s2.append(round(max(order-pgi-inv -ts,0)))
        elif ot == 'over' and pt !='exceeded': 
            gap_box_s2.append(round(max(mi*avg - pgi - inv -ts,0)))
        else:
            gap_box_s2.append(None)
    elif stg == 'shortage' and cus == 'KA':
        gap_box_s2.append(round(order-pgi))      
    else:
        gap_box_s2.append(None)

df['gap_box_s2'] = gap_box_s2
df['gap_box_s2'].sum() # 278,806 → 151,841

2225509.0

In [31]:
df[df['CusType']=='CRS'].gap_box_s2.sum()

153121.0

In [33]:
# scenario 3：OWA/avail_inventory  ==0  
    # normal GAP = min*avg - pgi - avail_inventory
    # under ordering GAP = order - pgi - avail_inventory
    # over ordering and normal pgi GAP = min*avg - pgi - avail_inventory
gap_box_s3= []

for ot,pt,stg,order,pgi,inv,mx,mi,avg,cus,ts in zip(df['order_type'],df['pgi_type'],df['shortage'],df['Order Qty'],df['ConfirmQty'],df['avail_inventory_box'],df['MaxDays'],df['MinDays'],df['avg_sales_box_day'],df['CusType'],df['intrans_inventory_box']):
    if inv <= 0:
        if stg == 'shortage' and cus == 'CRS':
            if ot == 'normal':
                gap_box_s3.append(round(max(mi*avg - pgi - inv -ts,0)))
            elif ot == 'under': 
                gap_box_s3.append(round(max(order-pgi-inv -ts,0)))
            elif ot == 'over' and pt !='exceeded': 
                gap_box_s3.append(round(max(mi*avg - pgi - inv -ts,0)))
            else:
                gap_box_s3.append(None)
        else:
            gap_box_s3.append(None)
    elif stg == 'shortage' and cus == 'KA':
        gap_box_s3.append(round(order-pgi))
    else:
        gap_box_s3.append(None)

df['gap_box_s3'] = gap_box_s3
df['gap_box_s3'].sum() # 65,147 ->  37,645

2110679.0

In [34]:
df[df['CusType']=='CRS'].gap_box_s3.sum()  #38291

38291.0

## * Align net price

In [35]:
sku_list = va_df[['Material','Net price']].groupby('Material').agg({'Net price':'mean'}).reset_index()
sku_list_ka = va_ka[['Material','Net price']].groupby('Material').agg({'Net price':'mean'}).reset_index()
sku_price_list = sku_list.append(sku_list_ka).drop_duplicates('Material').rename(columns = {'Material':'material'})

In [36]:
df_copy = df

In [37]:
df = df.drop(columns ='Net price')
df = pd.merge(df,sku_price_list, on='material',how='left')

## *2.2 Calculate gap can be fill by customer type

In [38]:
# GAP amount group by year, week, material and plant and customer type
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) #trim

sku_gap_fill = df[['year','week','material','Plnt','CusType','gap_box_s1','gap_box_s2','gap_box_s3']]\
                    .groupby(['year','week','material','Plnt','CusType'])\
                    .agg({'gap_box_s1':'sum','gap_box_s2':'sum','gap_box_s3':'sum'}).reset_index()
sku_gap_fill['key'] = sku_gap_fill['year'].map(str).str.cat([sku_gap_fill['week'].map(str),sku_gap_fill['material'].map(str),sku_gap_fill['Plnt'].map(str)],sep='')


# # Exceed by year,week,material,plant and to be disbribute by priority
sku_exceed_reallocate = df[['year','week','material','Plnt','exceed_pgi_box']]\
                    .groupby(['year','week','material','Plnt']).agg({'exceed_pgi_box':'sum'}).reset_index()
sku_exceed_reallocate['key'] =  sku_exceed_reallocate['year'].map(str).str.cat([sku_exceed_reallocate['week'].map(str),sku_exceed_reallocate['material'].map(str),sku_exceed_reallocate['Plnt'].map(str)],sep='')

sku_exceed_gap = pd.merge(sku_gap_fill,sku_exceed_reallocate[['key','exceed_pgi_box']],on=['key'],how='left')

In [39]:
# Calculate the gap can be fill by Customer Type

# 1. KA
gap_can_be_fill_s1 = []
gap_can_be_fill_s2 = []
gap_can_be_fill_s3 = []
updated_exceed_pgi_s1 = []
updated_exceed_pgi_s2 = []
updated_exceed_pgi_s3 = []

for key, cus, ex, g1, g2, g3 in zip(sku_exceed_gap['key'],sku_exceed_gap['CusType'],sku_exceed_gap['exceed_pgi_box'],\
                            sku_exceed_gap['gap_box_s1'],sku_exceed_gap['gap_box_s2'],sku_exceed_gap['gap_box_s3']):
    if cus == 'KA':
        gap_can_be_fill_s1.append(min(g1,ex))
        gap_can_be_fill_s2.append(min(g2,ex))
        gap_can_be_fill_s3.append(min(g3,ex))
        
        updated_exceed_pgi_s1.append(ex-min(g1,ex)) # update exceed_pgi
        updated_exceed_pgi_s2.append(ex-min(g2,ex))
        updated_exceed_pgi_s3.append(ex-min(g3,ex))
        
    else:
        gap_can_be_fill_s1.append(None)
        gap_can_be_fill_s2.append(None)
        gap_can_be_fill_s3.append(None)
        
        updated_exceed_pgi_s1.append(None) #只记录更新的exceed
        updated_exceed_pgi_s2.append(None)
        updated_exceed_pgi_s3.append(None)

sku_exceed_gap['gap_can_be_fill_s1'] = gap_can_be_fill_s1
sku_exceed_gap['gap_can_be_fill_s2'] = gap_can_be_fill_s2
sku_exceed_gap['gap_can_be_fill_s3'] = gap_can_be_fill_s3

sku_exceed_gap['updated_exceed_pgi_s1'] = updated_exceed_pgi_s1
sku_exceed_gap['updated_exceed_pgi_s2'] = updated_exceed_pgi_s2
sku_exceed_gap['updated_exceed_pgi_s3'] = updated_exceed_pgi_s3


# update exceed by KA
sku_exceed_updated = sku_exceed_gap[sku_exceed_gap['CusType']=='KA']  
sku_exceed_updated = sku_exceed_updated[['year','week','material','Plnt','updated_exceed_pgi_s1','updated_exceed_pgi_s2','updated_exceed_pgi_s3']]\
                    .groupby(['year','week','material','Plnt'])\
                    .agg({'updated_exceed_pgi_s1':'sum','updated_exceed_pgi_s2':'sum','updated_exceed_pgi_s3':'sum'}).reset_index()

sku_exceed_updated = sku_exceed_updated.rename(columns = {'updated_exceed_pgi_s1':'ka_updated_exceed_pgi_s1',\
                                                          'updated_exceed_pgi_s2':'ka_updated_exceed_pgi_s2',\
                                                          'updated_exceed_pgi_s3':'ka_updated_exceed_pgi_s3',})
# sku_exceed_gap = sku_exceed_gap.drop(columns = ['updated_exceed_pgi_s1','updated_exceed_pgi_s2','updated_exceed_pgi_s3'])
sku_exceed_gap_updated = pd.merge(sku_exceed_gap,sku_exceed_updated,on=['year','week','material','Plnt'],how='left')
sku_exceed_gap_updated['updated_exceed_s1'] = sku_exceed_gap_updated[['exceed_pgi_box','ka_updated_exceed_pgi_s1']].apply(lambda x: x['ka_updated_exceed_pgi_s1'] if x['ka_updated_exceed_pgi_s1']<x['exceed_pgi_box'] else x['exceed_pgi_box'],axis=1 )
sku_exceed_gap_updated['updated_exceed_s2'] = sku_exceed_gap_updated[['exceed_pgi_box','ka_updated_exceed_pgi_s2']].apply(lambda x: x['ka_updated_exceed_pgi_s2'] if x['ka_updated_exceed_pgi_s2']<x['exceed_pgi_box'] else x['exceed_pgi_box'],axis=1 )
sku_exceed_gap_updated['updated_exceed_s3'] = sku_exceed_gap_updated[['exceed_pgi_box','ka_updated_exceed_pgi_s3']].apply(lambda x: x['ka_updated_exceed_pgi_s3'] if x['ka_updated_exceed_pgi_s3']<x['exceed_pgi_box'] else x['exceed_pgi_box'],axis=1 )


# 2. CRS
gap_can_be_fill_s1 = []
gap_can_be_fill_s2 = []
gap_can_be_fill_s3 = []
updated_exceed_pgi_s1 = []
updated_exceed_pgi_s2 = []
updated_exceed_pgi_s3 = []

for key,cus,ex1,g1,f1,ex2,g2,f2,ex3,g3,f3 in zip(sku_exceed_gap_updated['key'],sku_exceed_gap_updated['CusType'],\
      sku_exceed_gap_updated['updated_exceed_s1'],sku_exceed_gap_updated['gap_box_s1'],sku_exceed_gap_updated['gap_can_be_fill_s1'],
      sku_exceed_gap_updated['updated_exceed_s2'],sku_exceed_gap_updated['gap_box_s2'],sku_exceed_gap_updated['gap_can_be_fill_s2'],
      sku_exceed_gap_updated['updated_exceed_s3'],sku_exceed_gap_updated['gap_box_s3'],sku_exceed_gap_updated['gap_can_be_fill_s3']):
    
    if cus == 'CRS':
        gap_can_be_fill_s1.append(min(g1,ex1))
        gap_can_be_fill_s2.append(min(g2,ex2))
        gap_can_be_fill_s3.append(min(g3,ex3))

        updated_exceed_pgi_s1.append(ex1-min(g1,ex1)) # update exceed_pgi
        updated_exceed_pgi_s2.append(ex2-min(g2,ex2))
        updated_exceed_pgi_s3.append(ex3-min(g3,ex3))
    else:
        gap_can_be_fill_s1.append(f1)
        gap_can_be_fill_s2.append(f2)
        gap_can_be_fill_s3.append(f3)

        updated_exceed_pgi_s1.append(ex1)
        updated_exceed_pgi_s2.append(ex2)
        updated_exceed_pgi_s3.append(ex3)

sku_exceed_gap_updated['gap_can_be_fill_s1'] = gap_can_be_fill_s1
sku_exceed_gap_updated['gap_can_be_fill_s2'] = gap_can_be_fill_s2
sku_exceed_gap_updated['gap_can_be_fill_s3'] = gap_can_be_fill_s3
sku_exceed_gap_updated['updated_exceed_pgi_s1'] = updated_exceed_pgi_s1
sku_exceed_gap_updated['updated_exceed_pgi_s2'] = updated_exceed_pgi_s2
sku_exceed_gap_updated['updated_exceed_pgi_s3'] = updated_exceed_pgi_s3

# drop assist columns
sku_exceed_gap_updated = sku_exceed_gap_updated.drop(columns = ['updated_exceed_s1','updated_exceed_s2','updated_exceed_s3',\
                                                        'ka_updated_exceed_pgi_s1','ka_updated_exceed_pgi_s2','ka_updated_exceed_pgi_s3'])

In [40]:
sku_gap_fill = sku_exceed_gap_updated

In [41]:
# Calculate the gap fill index 
sku_gap_fill['gap_fill_index_s1'] = (sku_gap_fill['gap_can_be_fill_s1']/sku_gap_fill['gap_box_s1']).replace(np.inf, 0)
sku_gap_fill['gap_fill_index_s2'] = (sku_gap_fill['gap_can_be_fill_s2']/sku_gap_fill['gap_box_s2']).replace(np.inf, 0)
sku_gap_fill['gap_fill_index_s3'] = (sku_gap_fill['gap_can_be_fill_s3']/sku_gap_fill['gap_box_s3']).replace(np.inf, 0)

In [42]:
df['Order Qty'].sum()

45541778.484000005

In [43]:
# Map index back to df, by year/week/material/plant/customer type
df1 = pd.merge(df, sku_gap_fill[['year','week','material','Plnt','CusType','gap_fill_index_s1','gap_fill_index_s2','gap_fill_index_s3']],
               on=['year','week','material','Plnt','CusType'],how='left')

In [44]:
# Calculate gap can be fill value
gap_box_can_be_fill_s1 = []
gap_box_can_be_fill_s2 = []
gap_box_can_be_fill_s3 = []
gap_can_be_fill_value_s1 = []
gap_can_be_fill_value_s2 = []
gap_can_be_fill_value_s3 = []


for ot,pt,stg,g1,g2,g3,ix1,ix2,ix3,price,cus in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],
                                                    df1['gap_box_s1'],df1['gap_box_s2'],df1['gap_box_s3'],
                                df1['gap_fill_index_s1'],df1['gap_fill_index_s2'],df1['gap_fill_index_s3'],df1['Net price'],df1['CusType']):
   
    if stg == 'shortage' and pt != 'exceeded':
        # calculate gap box can be fill
        gap_box_can_be_fill_s1.append(g1*ix1)
        gap_box_can_be_fill_s2.append(g2*ix2)
        gap_box_can_be_fill_s3.append(g3*ix3)
        
        # calculate gap box value
        gap_can_be_fill_value_s1.append(g1*ix1*price)
        gap_can_be_fill_value_s2.append(g2*ix2*price)
        gap_can_be_fill_value_s3.append(g3*ix3*price)
        
    elif stg == 'shortage' and cus == 'KA': #do allocation only for shortage sku
        # calculate gap box can be fill
        gap_box_can_be_fill_s1.append(g1*ix1)
        gap_box_can_be_fill_s2.append(g2*ix2)
        gap_box_can_be_fill_s3.append(g3*ix3)
        
        # calculate gap box value
        gap_can_be_fill_value_s1.append(g1*ix1*price)
        gap_can_be_fill_value_s2.append(g2*ix2*price)
        gap_can_be_fill_value_s3.append(g3*ix3*price)
        
    else:
        gap_box_can_be_fill_s1.append(None)
        gap_box_can_be_fill_s2.append(None)
        gap_box_can_be_fill_s3.append(None)
        
        gap_can_be_fill_value_s1.append(None)
        gap_can_be_fill_value_s2.append(None)
        gap_can_be_fill_value_s3.append(None)
        
df1['gap_box_can_be_fill_s1'] = gap_box_can_be_fill_s1
df1['gap_box_can_be_fill_s2'] = gap_box_can_be_fill_s2
df1['gap_box_can_be_fill_s3'] = gap_box_can_be_fill_s3
df1['gap_can_be_fill_value_s1'] = gap_can_be_fill_value_s1
df1['gap_can_be_fill_value_s2'] = gap_can_be_fill_value_s2
df1['gap_can_be_fill_value_s3'] = gap_can_be_fill_value_s3

# Calculate the sum
df1[['gap_box_can_be_fill_s1','gap_box_can_be_fill_s2','gap_box_can_be_fill_s3',
     'gap_can_be_fill_value_s1','gap_can_be_fill_value_s2','gap_can_be_fill_value_s3']].sum().astype(int)

gap_box_can_be_fill_s1        426706
gap_box_can_be_fill_s2        383547
gap_box_can_be_fill_s3        349995
gap_can_be_fill_value_s1    64378343
gap_can_be_fill_value_s2    57961679
gap_can_be_fill_value_s3    53473356
dtype: int32

In [45]:
df_dp=df1

## *2.3 Calculate reallocated exceed pgi to fill the gap 

In [46]:
# GAP can be fill and exceed pgi amount group by year, week, material and plant and *CusType
        # Applied in adjusted order
sku_exceed_reallocate = df1[['year','week','material','Plnt','exceed_pgi_box'
                             ,'gap_box_can_be_fill_s1','gap_box_can_be_fill_s2','gap_box_can_be_fill_s3']]\
                                .groupby(['year','week','material','Plnt'])\
                                 .agg({'gap_box_can_be_fill_s1':'sum','gap_box_can_be_fill_s2':'sum','gap_box_can_be_fill_s3':'sum','exceed_pgi_box':'sum'}).reset_index()

sku_exceed_reallocate['exceed_pgi_box'] = sku_exceed_reallocate['exceed_pgi_box'].astype(float)

# calculate exceed index 
sku_exceed_reallocate['exceed_fill_index_s1'] = (sku_exceed_reallocate['gap_box_can_be_fill_s1']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_fill_index_s2'] = (sku_exceed_reallocate['gap_box_can_be_fill_s2']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_fill_index_s3'] = (sku_exceed_reallocate['gap_box_can_be_fill_s3']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)

sku_exceed_reallocate['exceed_can_not_be_allocated_s1'] = sku_exceed_reallocate['exceed_pgi_box']-sku_exceed_reallocate['gap_box_can_be_fill_s1']
sku_exceed_reallocate['exceed_can_not_be_allocated_s2'] = sku_exceed_reallocate['exceed_pgi_box']-sku_exceed_reallocate['gap_box_can_be_fill_s2']
sku_exceed_reallocate['exceed_can_not_be_allocated_s3'] = sku_exceed_reallocate['exceed_pgi_box']-sku_exceed_reallocate['gap_box_can_be_fill_s3']

sku_exceed_reallocate['exceed_back_index_s1'] = (sku_exceed_reallocate['exceed_can_not_be_allocated_s1']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_back_index_s2'] = (sku_exceed_reallocate['exceed_can_not_be_allocated_s2']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)
sku_exceed_reallocate['exceed_back_index_s3'] = (sku_exceed_reallocate['exceed_can_not_be_allocated_s3']/sku_exceed_reallocate['exceed_pgi_box']).replace(np.inf,0)


# Map index back to df1
df1 = pd.merge(df1, sku_exceed_reallocate[['year','week','material','Plnt','exceed_fill_index_s1','exceed_fill_index_s2','exceed_fill_index_s3','exceed_back_index_s1','exceed_back_index_s2','exceed_back_index_s3']],
                           on=['year','week','material','Plnt'],how='left')

In [288]:
df1['exceed_fill_index_s1'] = df1[['CusType','exceed_fill_index_s1']].apply(lambda x: 0 if x['CusType']=='KA' else x['exceed_fill_index_s1'],axis=1)
df1['exceed_fill_index_s2'] = df1[['CusType','exceed_fill_index_s2']].apply(lambda x: 0 if x['CusType']=='KA' else x['exceed_fill_index_s2'],axis=1)
df1['exceed_fill_index_s3'] = df1[['CusType','exceed_fill_index_s3']].apply(lambda x: 0 if x['CusType']=='KA' else x['exceed_fill_index_s3'],axis=1)

In [47]:
df1['pgi_after_allocation_s1'] = df1['exceed_pgi_box']*df1['exceed_back_index_s1']
df1['pgi_after_allocation_s2'] = df1['exceed_pgi_box']*df1['exceed_back_index_s2']
df1['pgi_after_allocation_s3'] = df1['exceed_pgi_box']*df1['exceed_back_index_s3']

# drop assistant columns
df1 = df1.drop(columns = ['exceed_back_index_s1','exceed_back_index_s2','exceed_back_index_s3'])

## 2.4 Calculate adjusted order qty

In [50]:
adjusted_order_qty = []

for ot,pt,stg,order,inv,mx,avg,sale,cus,ts in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['Order Qty'],
                                      df1['avail_inventory_box'],df1['MaxDays'],df1['avg_sales_box_day'],df1['sale_month'],df1['CusType'],df1['intrans_inventory_box']):
    if stg == 'shortage' and ot == 'over' and cus == 'CRS':
        if inv + ts > mx*avg: # Case 1: conside max boundary
            aj_order_a = 0
        else: # inv +ts <= mx*avg
            aj_order_a = mx*avg - inv -ts
        if order + inv  > sale: # Case: consider following month sale
            aj_order_b = sale - inv 
        else:
            aj_order_b = order
        adjusted_order_qty.append(max(aj_order_a, aj_order_b))
    else:
        adjusted_order_qty.append(order) # no touch on KA's order
        
df1['adjusted_order_qty'] = adjusted_order_qty
df1['adjusted_order_qty'].sum().astype(int) #23,539,577 → 45,431,958

42789217

In [51]:
adjusted_order_qty_s1 = []
adjusted_order_qty_s2 = []
adjusted_order_qty_s3 = []


for ot,pt,stg,order,cus,aj,p1,p2,p3 in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['Order Qty'],df1['CusType'],
                df1['adjusted_order_qty'],df1['pgi_after_allocation_s1'],df1['pgi_after_allocation_s2'],df1['pgi_after_allocation_s3']):
    if stg == 'shortage' and ot == 'over' and cus == 'CRS':
        adjusted_order_qty_s1.append(max(aj, order-p1))
        adjusted_order_qty_s2.append(max(aj, order-p2))
        adjusted_order_qty_s3.append(max(aj, order-p3))

    else:
        adjusted_order_qty_s1.append(order) # no touch on KA's order
        adjusted_order_qty_s2.append(order)
        adjusted_order_qty_s3.append(order)
        
        
df1['adjusted_order_qty_s1'] = adjusted_order_qty_s1
df1['adjusted_order_qty_s2'] = adjusted_order_qty_s2
df1['adjusted_order_qty_s3'] = adjusted_order_qty_s3

df1['adjusted_order_qty_s1'].sum().astype(int) 

43576600

## 2.5 Calculate adjusted pgi qty

In [52]:
# fill na with 0 for new cols
qualitative,quantitative = qual_quant_features(df1)
df1[quantitative] = df1[quantitative].fillna(0)

adjusted_confirm_qty_s1 = []
adjusted_confirm_qty_s2 = []
adjusted_confirm_qty_s3 = []

for ot,pt,stg,aj1,aj2,aj3,pgi,gap1,gap2,gap3,ex, ex_ix1,ex_ix2,ex_ix3,cus \
    in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['adjusted_order_qty_s1'],df1['adjusted_order_qty_s2'],df1['adjusted_order_qty_s3'],df1['ConfirmQty'],
           df1['gap_box_can_be_fill_s1'],df1['gap_box_can_be_fill_s2'],df1['gap_box_can_be_fill_s3'],
           df1['exceed_pgi_box'],df1['exceed_fill_index_s1'],df1['exceed_fill_index_s2'],df1['exceed_fill_index_s3'],df1['CusType']):
        
        if cus == 'CRS':
            if stg == 'normal':
                adjusted_confirm_qty_s1.append(pgi)
                adjusted_confirm_qty_s2.append(pgi)
                adjusted_confirm_qty_s3.append(pgi)

            elif stg == 'shortage' and pt != 'exceeded':
                adjusted_confirm_qty_s1.append(min(pgi+gap1, aj1))
                adjusted_confirm_qty_s2.append(min(pgi+gap2, aj2))
                adjusted_confirm_qty_s3.append(min(pgi+gap3, aj3))

            elif stg == 'shortage' and pt == 'exceeded':
                adjusted_confirm_qty_s1.append(min(pgi-ex*ex_ix1, aj1))
                adjusted_confirm_qty_s2.append(min(pgi-ex*ex_ix2, aj2))
                adjusted_confirm_qty_s3.append(min(pgi-ex*ex_ix3, aj3))
                
        else: # cus =='KA'
            if stg != 'shortage':
                adjusted_confirm_qty_s1.append(pgi)
                adjusted_confirm_qty_s2.append(pgi)
                adjusted_confirm_qty_s3.append(pgi)
                
            elif stg == 'shortage':
                adjusted_confirm_qty_s1.append(pgi+gap1)
                adjusted_confirm_qty_s2.append(pgi+gap2)
                adjusted_confirm_qty_s3.append(pgi+gap3)


df1['adjusted_confirm_qty_s1'] = adjusted_confirm_qty_s1
df1['adjusted_confirm_qty_s2'] = adjusted_confirm_qty_s2
df1['adjusted_confirm_qty_s3'] = adjusted_confirm_qty_s3

df1[['adjusted_confirm_qty_s1','adjusted_confirm_qty_s2','adjusted_confirm_qty_s3']].sum().astype(int)

adjusted_confirm_qty_s1    40514606
adjusted_confirm_qty_s2    40459773
adjusted_confirm_qty_s3    40412349
dtype: int32

# 3 Calculate the benefit

## 3.1 CFR

In [53]:
df_cfr = df1[['order_type','pgi_type','shortage','CusType','Order Qty','ConfirmQty','adjusted_order_qty_s1','adjusted_order_qty_s2','adjusted_order_qty_s3','adjusted_confirm_qty_s1','adjusted_confirm_qty_s2','adjusted_confirm_qty_s3']]

df_cfr_groupby_shortage = df_cfr.groupby(['shortage','CusType']).sum().reset_index()

df_cfr_groupby_shortage['original_cfr'] = df_cfr_groupby_shortage['ConfirmQty']/df_cfr_groupby_shortage['Order Qty']*100
df_cfr_groupby_shortage['adjusted_cfr_s1'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s1']/df_cfr_groupby_shortage['adjusted_order_qty_s1']*100
df_cfr_groupby_shortage['adjusted_cfr_s2'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s2']/df_cfr_groupby_shortage['adjusted_order_qty_s2']*100
df_cfr_groupby_shortage['adjusted_cfr_s3'] = df_cfr_groupby_shortage['adjusted_confirm_qty_s3']/df_cfr_groupby_shortage['adjusted_order_qty_s3']*100

cfr = df_cfr_groupby_shortage[['shortage','CusType','original_cfr','adjusted_cfr_s1','adjusted_cfr_s2','adjusted_cfr_s3']]
cfr

,shortage,CusType,original_cfr,adjusted_cfr_s1,adjusted_cfr_s2,adjusted_cfr_s3
0,normal,CRS,99.708370,99.708370,99.708370,99.708370
1,normal,KA,99.226985,99.226985,99.226985,99.226985
2,shortage,CRS,90.274161,86.125604,85.923199,85.709271
3,shortage,KA,80.208761,83.422698,83.422698,83.422698


## 3.2 Sales 
#### CRS by allocation_value* index , KA by allocation_value * CFR% improvement

In [54]:
drive_sale_s1 = []
drive_sale_s2 = []
drive_sale_s3 = []

s1_index = 0.25
s2_index = 0.5
s3_index = 1
ka_index = 1 # (cfr[ (cfr['shortage']=='shortage') & (cfr['CusType']=='KA')].adjusted_cfr_s1  -cfr[ (cfr['shortage']=='shortage') & (cfr['CusType']=='KA')].original_cfr)/100

for ot,pt,stg,cus, gv1,gv2,gv3 in zip(df1['order_type'],df1['pgi_type'],df1['shortage'],df1['CusType'],
                                        df1['gap_can_be_fill_value_s1'],df1['gap_can_be_fill_value_s2'],df1['gap_can_be_fill_value_s3']):
        
        if stg == 'shortage' and pt != 'exceed'and cus =='CRS':
            drive_sale_s1.append(s1_index*gv1)
            drive_sale_s2.append(s2_index*gv2)
            drive_sale_s3.append(s3_index*gv3)  
            
        elif stg == 'shortage' and cus =='KA':
            drive_sale_s1.append(ka_index*gv1)
            drive_sale_s2.append(ka_index*gv2)
            drive_sale_s3.append(ka_index*gv3)
            
        else: 
            drive_sale_s1.append(None)
            drive_sale_s2.append(None)
            drive_sale_s3.append(None)
            
df1['drive_sale_s1']=drive_sale_s1
df1['drive_sale_s2']=drive_sale_s2
df1['drive_sale_s3']=drive_sale_s3
df1[['drive_sale_s1','drive_sale_s2','drive_sale_s3']].sum().astype(int)

drive_sale_s1    54707003
drive_sale_s2    54722451
drive_sale_s3    53473356
dtype: int32

In [293]:
#df1.to_csv(r'C:\Users\the7490\Downloads\crs_processed_shortage_crs+kasap_94_transit_1.8_ka_excl_61_tag_incl61_exceed_back.csv',encoding='utf-8',index = None)